In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv', index_col='essay_id')
test = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv', index_col='essay_id')

submission = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv')

In [ ]:
train = train.sort_values('score')

# 並べ替え

In [ ]:
train

In [ ]:

# textBlobを用いて、スペルミスの数を数えます

from textblob import TextBlob
import nltk
nltk.download('punkt')
#　nltk.download('wordnet')
train['spelling_errors'] = train['full_text'].apply(lambda x: len(TextBlob(x).correct().words) - len(TextBlob(x).words))
print(train.head())

print(train['spelling_errors'].groupby(train['score']).mean())


In [ ]:
'''
# scoreで正規表現からスペルミスの数を比較してみる
train['spelling_errors'] = train.full_text.str.count(r'\b\w{20,}\b')
train['spelling_errors'].groupby(train['score']).mean()


'''


In [ ]:
train

In [ ]:
test['spelling_errors'] = test['full_text'].apply(lambda x: len(TextBlob(x).correct().words) - len(TextBlob(x).words))

test


In [ ]:
# 文章ごとの語の数 
train['word_count'] = train.full_text.str.count(r'\w+')
test['word_count'] = test.full_text.str.count(r'\w+')

train

In [ ]:
# 語の数を比較してみる

train['word_count'].groupby(train['score']).mean()

In [ ]:
!pip install textstat

In [ ]:

import textstat

# Flesch-Kincaid Grade Levelを特徴量として追加する

train['flesch_kincaid_grade'] = train.full_text.apply(textstat.flesch_kincaid_grade)
test['flesch_kincaid_grade'] = test.full_text.apply(textstat.flesch_kincaid_grade)
train


In [ ]:
train['flesch_kincaid_grade'].groupby(train['score']).mean()

In [ ]:
'''
from textblob import TextBlob

# 感情分析のスコアを特徴量として追加する

train['sentiment'] = train.full_text.apply(lambda x: TextBlob(x).sentiment.polarity)
test['sentiment'] = test.full_text.apply(lambda x: TextBlob(x).sentiment.polarity)

print(train['sentiment'].groupby(train['score']).mean())
train
'''

In [ ]:
'''
!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# 感情分析のスコアを特徴量として追加する

analyzer = SentimentIntensityAnalyzer()

train['vader_sentiment'] = train.full_text.apply(lambda x: analyzer.polarity_scores(x)['compound'])

test['vader_sentiment'] = test.full_text.apply(lambda x: analyzer.polarity_scores(x)['compound'])

'''

In [ ]:
test

In [ ]:
train

In [ ]:
#　スペルミスの割合
train['spelling_errors_rate'] = train['spelling_errors'] / train['word_count']
test['spelling_errors_rate'] = test['spelling_errors'] / test['word_count']

train

In [ ]:
# スペルミスの割合とスコアの関係を見ます

train[['spelling_errors_rate', 'score']].groupby(train['score']).mean()


In [ ]:
# 誤字脱字のある単語を除いてみます

import nltk
nltk.download('words')

from nltk.corpus import words

train['corrected_text'] = train['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word in words.words()]))

test['corrected_text'] = test['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word in words.words()]))

train

In [ ]:
# nltkヲ使って文章をトークン化する
import nltk

nltk.download('punkt')

from nltk.tokenize import word_tokenize

data['tokens'] = data['corrected_text'].apply(word_tokenize)
test['tokens'] = test['corrected_text'].apply(word_tokenize)
data

# ストップワードを取り除く

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
test['tokens'] = test['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

In [ ]:
# スペルミスのない正しい文章をストップワードを除いてみます

nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

data['corrected_text'] = data['corrected_text'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

test['corrected_text'] = test['corrected_text'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

data

In [ ]:
'''

# 文章をトークン化する
import nltk

nltk.download('punkt')

from nltk.tokenize import word_tokenize

train['tokens'] = train['full_text'].apply(word_tokenize)
test['tokens'] = test['full_text'].apply(word_tokenize)

# ストップワードを取り除く

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
train['tokens'] = train['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
test['tokens'] = test['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

'''

In [ ]:
# ストップワードを取り除いた後の語の数を特徴量として追加する

train['word_count_no_stopwords'] = train['tokens'].apply(len)
test['word_count_no_stopwords'] = test['tokens'].apply(len)
train

In [ ]:
# ストップワードを取り除いた後の語の数とスコアの関係を見ます

train[['word_count_no_stopwords', 'score']].groupby(train['score']).mean()


In [ ]:
test['word_count_no_stopwords'] = test['tokens'].apply(len)
test

In [ ]:
# 語彙という意味で、tokenを集合にして、数値化したものを特徴量にします

train['vocabulary'] = train['tokens'].apply(set)

test['vocabulary'] = test['tokens'].apply(set)

train


# 語彙の数

train['vocabulary_count'] = train['vocabulary'].apply(len)

train


test['vocabulary_count'] = test['vocabulary'].apply(len)

In [ ]:
# scoreで語彙の数を比較してみる

train['vocabulary_count'].groupby(train['score']).mean()


In [ ]:
train

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(train['score']).astype(int)
y

In [ ]:
train

In [ ]:
train.drop(['full_text', 'tokens', 'vocabulary', 'score'], axis = 1, inplace = True)

In [ ]:
type(train)

In [ ]:
train

In [ ]:
test

In [ ]:
test.drop(['full_text', 'tokens', 'vocabulary'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,y, shuffle = True)

In [ ]:
# lightgbmを使ってモデルを作成します

import lightgbm as lgb

lgb_train = lgb.Dataset(X_train, y_train)

lgb_valid = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

params = {
    'objective': 'multiclass',
    'num_class': 6,
    'metric': 'multi_logloss',
    'verbosity': -1
}

model_0 = lgb.train(params, lgb_train, valid_sets=lgb_valid, num_boost_round=1000, early_stopping_rounds=10)





In [ ]:
# テストデータを予測します

y_pred = model_0.predict(test)

y_pred


In [ ]:

# 提出用のデータフレームを作成します

submission['score'] = le.inverse_transform(np.argmax(y_pred, axis=1)).astype(int)

submission



In [ ]:
# 提出用のデータフレームを保存します

submission.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_1 = RandomForestClassifier(n_estimators=100, random_state=42)
model_1.fit(train, y)

model_2 = LineerRegression()
model_2.fit(X_train, y_train)

In [ ]:
y_pred = model_2.predict(X_valid)
y_pred = le.inverse_transform(y_pred)


In [ ]:
# cohen kappa scoreを用いて評価します

from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_valid, y_pred, weights='quadratic')

In [ ]:
test.drop(['full_text', 'tokens', 'vocabulary'], axis = 1, inplace = True)

In [ ]:
test

In [ ]:
prediction = model_2.predict(test)

In [ ]:
pred = le.inverse_transform(prediction)

In [ ]:
pred

In [ ]:
submission['score'] = pred

submission

In [ ]:
# submission.to_csv('submission.csv', index = False)